In [1]:
import z3
import json
import pprint
import numpy as np
import pandas as pd

with open ('simple.json') as file:
    data = json.load(file)

#------------SETUP-------------------------------

# List of Resource Types
resourceTypes = set([j for i in data for j in data[i]['resources'] ])

# Initialize data structures
nodeList = list()
appList = list()
nlappList = list() # List of apps that are dependencies
prAppList = list() # List of primary apps that have dependencies
a2a = []
n2n = []

# Node and App lists
for i in data:
    if data[i]['type']=='node':
        nodeList.append(i)
    if data[i]['type']=='app':
        appList.append(i)
        if data[i]['dependencies']['nonlocal']:
            prAppList.append(i)
            for dep in data[i]['dependencies']['nonlocal']:
                nlappList.append(dep)
print("Dependency apps: %s" %(nlappList))
print("Primary apps: %s" %(prAppList))
# app to app dependency matrix            
for app in appList:
    nl_deps = data[app]['dependencies']['nonlocal']
    deps = []
    for dep in appList:
        if dep in nl_deps:
            deps.append(1)
        else: 
            deps.append(0)
    a2a.append(deps)
a2a_df = pd.DataFrame(a2a, index=appList, columns=appList)        

# node to node latency matrix
n2n = []
for node in nodeList:
    id = data[node]['id']
    temp = []    
    for n2 in nodeList:
        id2 = data[n2]['id']
        lat = data[node]['latencies']['n%s%s' %(id, id2)]
        temp.append(lat)
    n2n.append(temp)
n2n_df = pd.DataFrame(n2n, index=nodeList, columns=nodeList)  

# node resource matrix
rpn= [[data[j]['resources'][k] for j in nodeList] for k in resourceTypes]
rpn_df = pd.DataFrame(rpn, index=resourceTypes, columns=nodeList)     
    
# app resource matrix
rpa= [[data[i]['resources'][k] for i in appList] for k in resourceTypes]
rpa_df = pd.DataFrame(rpa, index=resourceTypes, columns=appList)    
print(rpa_df.loc['memory', 'APP1'])
print(type(rpa_df.loc['memory', 'APP1']))

# app to node placment matrix
a2n = [ [ z3.Real("a%s%s" % (i+1, j+1)) for j in range(len(nodeList)) ]
            for i in range(len(appList))]
a2n_df = pd.DataFrame(a2n, index=appList, columns=nodeList) 

print("n2n: \n%s" %n2n_df)
print("a2a: \n%s" %a2a_df)
print("rpn: \n%s" %rpn_df)
print("rpa: \n%s" %rpa_df)

print("a2n: \n%s" %a2n_df)

Dependency apps: ['APP2']
Primary apps: ['APP1']
256
<class 'numpy.int64'>
n2n: 
       FSSN2  FSSN4  FSSN1  FSSN3  FSSN5
FSSN2      0      6      3      5      7
FSSN4      6      0      5      7      9
FSSN1      3      5      0      4      6
FSSN3      5      7      4      0      8
FSSN5      7      9      6      8      0
a2a: 
      APP1  APP2
APP1     0     1
APP2     0     0
rpn: 
        FSSN2  FSSN4  FSSN1  FSSN3  FSSN5
memory    512    512   1024    512    512
rpa: 
        APP1  APP2
memory   256   128
a2n: 
     FSSN2 FSSN4 FSSN1 FSSN3 FSSN5
APP1   a11   a12   a13   a14   a15
APP2   a21   a22   a23   a24   a25


In [2]:
a2n_df

,FSSN2,FSSN4,FSSN1,FSSN3,FSSN5
APP1,a11,a12,a13,a14,a15
APP2,a21,a22,a23,a24,a25


In [3]:
n2n_df

,FSSN2,FSSN4,FSSN1,FSSN3,FSSN5
FSSN2,0,6,3,5,7
FSSN4,6,0,5,7,9
FSSN1,3,5,0,4,6
FSSN3,5,7,4,0,8
FSSN5,7,9,6,8,0


In [4]:
a2a_df

,APP1,APP2
APP1,0,1
APP2,0,0


In [5]:
rpn_df

,FSSN2,FSSN4,FSSN1,FSSN3,FSSN5
memory,512,512,1024,512,512


In [6]:
rpa_df

,APP1,APP2
memory,256,128


In [7]:
#------------Constraints-------------------------------
# variables in a2n matrix must be binary

a2n_domain = [z3.Or(a2n_df.loc[app, node]==0, a2n_df.loc[app, node]==1) \
 for app in nlappList \
 for node in nodeList]

# An intial deployment of apps that have dependencies
a2n_df.loc['APP1'] = 1
#a2n_df.loc['APP1', 'FSSN2'] = 1
#a2n_df.loc['APP1', 'FSSN3'] = 1
print (a2n_df)
a2n_domain


     FSSN2 FSSN4 FSSN1 FSSN3 FSSN5
APP1     1     1     1     1     1
APP2   a21   a22   a23   a24   a25


[Or(a21 == 0, a21 == 1),
 Or(a22 == 0, a22 == 1),
 Or(a23 == 0, a23 == 1),
 Or(a24 == 0, a24 == 1),
 Or(a25 == 0, a25 == 1)]

In [8]:
# Max capacity constraint
# Resources required by apps deployed on a node cannot exceed that nodes resources
app_util = rpa_df.dot(a2n_df)
print(app_util)
node_util = rpn_df - app_util
print(node_util)
rsrc_constraint = [node_util.loc[rsrc,node]>=0 \
                   for rsrc in resourceTypes \
                   for node in nodeList]
rsrc_constraint

                FSSN2          FSSN4          FSSN1          FSSN3  \
memory  256 + 128*a21  256 + 128*a22  256 + 128*a23  256 + 128*a24   

                FSSN5  
memory  256 + 128*a25  
                        FSSN2                  FSSN4                   FSSN1  \
memory  512 - (256 + 128*a21)  512 - (256 + 128*a22)  1024 - (256 + 128*a23)   

                        FSSN3                  FSSN5  
memory  512 - (256 + 128*a24)  512 - (256 + 128*a25)  


[512 - (256 + 128*a21) >= 0,
 512 - (256 + 128*a22) >= 0,
 1024 - (256 + 128*a23) >= 0,
 512 - (256 + 128*a24) >= 0,
 512 - (256 + 128*a25) >= 0]

In [9]:
# Must Deploy Dependencies constraint
# Assuming the primary app is deployed, all of it's dependencies must be deployed
mdd = [a2n_df.loc[dep].sum() > 0 for dep in nlappList]
mdd

[a21 + a22 + a23 + a24 + a25 > 0]

In [10]:
print([n2n_df.loc[src,dst] for src in nodeList for dst in nodeList])

[0, 6, 3, 5, 7, 6, 0, 5, 7, 9, 3, 5, 0, 4, 6, 5, 7, 4, 0, 8, 7, 9, 6, 8, 0]


In [11]:
n2n_df

,FSSN2,FSSN4,FSSN1,FSSN3,FSSN5
FSSN2,0,6,3,5,7
FSSN4,6,0,5,7,9
FSSN1,3,5,0,4,6
FSSN3,5,7,4,0,8
FSSN5,7,9,6,8,0


In [12]:
# Latency 
numNodes = len(nodeList) # number of FSSNs
markings = list() # The equations for each possible placement of the dependencies

for srcNode in nodeList:
    print (srcNode)
    for prApp in prAppList:
        if a2n_df.loc[app, srcNode] == 1:            
            ### ITERATE THROUGH ALL DEPENDENCIES
            # iterate through the 2^numNodes potential dependency placements
            for i in range(2**numNodes):
                mark = format(i, 'b').zfill(numNodes) # express placment in binary    
                #print(mark)
                #if dependency is not placed skip.
                if mark == format(0,'b').zfill(numNodes):
                    continue

                marking = list() # A placment equation e.g (1 - a11)*(1 - a12)*a13
                latency = [] # latencies between node hosting primary app and all nodes hosting the dependency for a particular placement. 
                print ("-----NEW MARKING-----")
                # iterate through nodes using the binary placement to generate marking equation
                for i, node in enumerate(nodeList):
                    #print ("mark %s" %mark)
                    #print ("i %s " %i)
                    #print ("mark[i] %s" %type(int(mark[i]))) 
                    # add marking to list and if binary has a 1, add latency to list of possible latencies
                    if int(mark[i]) == 1:
                        marking.append(a2n_df.loc['APP2', node])
                        latency.append(n2n_df.loc[srcNode, node])
                    if int(mark[i]) == 0:
                        marking.append(1 - a2n_df.loc['APP2',node])
                print("latency %s" %latency)
                print("min latency %s" %min(latency))
                print("marking: %s" %marking)
                print("marking * latency: %s" %(z3.Product(marking) * min(latency).item()))
                markings.append(z3.Product(marking) * min(latency).item())

print(markings)
latency_obj = z3.Sum(markings)
pprint.pprint(latency_obj)

    



FSSN2
-----NEW MARKING-----
latency [7]
min latency 7
marking: [1 - a21, 1 - a22, 1 - a23, 1 - a24, a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*(1 - a24)*a25*7
-----NEW MARKING-----
latency [5]
min latency 5
marking: [1 - a21, 1 - a22, 1 - a23, a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*a24*(1 - a25)*5
-----NEW MARKING-----
latency [5, 7]
min latency 5
marking: [1 - a21, 1 - a22, 1 - a23, a24, a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*a24*a25*5
-----NEW MARKING-----
latency [3]
min latency 3
marking: [1 - a21, 1 - a22, a23, 1 - a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*a23*(1 - a24)*(1 - a25)*3
-----NEW MARKING-----
latency [3, 7]
min latency 3
marking: [1 - a21, 1 - a22, a23, 1 - a24, a25]
marking * latency: (1 - a21)*(1 - a22)*a23*(1 - a24)*a25*3
-----NEW MARKING-----
latency [3, 5]
min latency 3
marking: [1 - a21, 1 - a22, a23, a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*a23*a24*(1 - a25)*3
-----NEW MARKING-----
latency [

marking * latency: a21*a22*(1 - a23)*(1 - a24)*(1 - a25)*0
-----NEW MARKING-----
latency [6, 0, 9]
min latency 0
marking: [a21, a22, 1 - a23, 1 - a24, a25]
marking * latency: a21*a22*(1 - a23)*(1 - a24)*a25*0
-----NEW MARKING-----
latency [6, 0, 7]
min latency 0
marking: [a21, a22, 1 - a23, a24, 1 - a25]
marking * latency: a21*a22*(1 - a23)*a24*(1 - a25)*0
-----NEW MARKING-----
latency [6, 0, 7, 9]
min latency 0
marking: [a21, a22, 1 - a23, a24, a25]
marking * latency: a21*a22*(1 - a23)*a24*a25*0
-----NEW MARKING-----
latency [6, 0, 5]
min latency 0
marking: [a21, a22, a23, 1 - a24, 1 - a25]
marking * latency: a21*a22*a23*(1 - a24)*(1 - a25)*0
-----NEW MARKING-----
latency [6, 0, 5, 9]
min latency 0
marking: [a21, a22, a23, 1 - a24, a25]
marking * latency: a21*a22*a23*(1 - a24)*a25*0
-----NEW MARKING-----
latency [6, 0, 5, 7]
min latency 0
marking: [a21, a22, a23, a24, 1 - a25]
marking * latency: a21*a22*a23*a24*(1 - a25)*0
-----NEW MARKING-----
latency [6, 0, 5, 7, 9]
min latency 0
ma

marking * latency: a21*a22*(1 - a23)*a24*a25*0
-----NEW MARKING-----
latency [5, 7, 4]
min latency 4
marking: [a21, a22, a23, 1 - a24, 1 - a25]
marking * latency: a21*a22*a23*(1 - a24)*(1 - a25)*4
-----NEW MARKING-----
latency [5, 7, 4, 8]
min latency 4
marking: [a21, a22, a23, 1 - a24, a25]
marking * latency: a21*a22*a23*(1 - a24)*a25*4
-----NEW MARKING-----
latency [5, 7, 4, 0]
min latency 0
marking: [a21, a22, a23, a24, 1 - a25]
marking * latency: a21*a22*a23*a24*(1 - a25)*0
-----NEW MARKING-----
latency [5, 7, 4, 0, 8]
min latency 0
marking: [a21, a22, a23, a24, a25]
marking * latency: a21*a22*a23*a24*a25*0
FSSN5
-----NEW MARKING-----
latency [0]
min latency 0
marking: [1 - a21, 1 - a22, 1 - a23, 1 - a24, a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*(1 - a24)*a25*0
-----NEW MARKING-----
latency [8]
min latency 8
marking: [1 - a21, 1 - a22, 1 - a23, a24, 1 - a25]
marking * latency: (1 - a21)*(1 - a22)*(1 - a23)*a24*(1 - a25)*8
-----NEW MARKING-----
latency [8, 0]
min latenc

(1 - a21)*(1 - a22)*(1 - a23)*(1 - a24)*a25*7 +
(1 - a21)*(1 - a22)*(1 - a23)*a24*(1 - a25)*5 +
(1 - a21)*(1 - a22)*(1 - a23)*a24*a25*5 +
(1 - a21)*(1 - a22)*a23*(1 - a24)*(1 - a25)*3 +
(1 - a21)*(1 - a22)*a23*(1 - a24)*a25*3 +
(1 - a21)*(1 - a22)*a23*a24*(1 - a25)*3 +
(1 - a21)*(1 - a22)*a23*a24*a25*3 +
(1 - a21)*a22*(1 - a23)*(1 - a24)*(1 - a25)*6 +
(1 - a21)*a22*(1 - a23)*(1 - a24)*a25*6 +
(1 - a21)*a22*(1 - a23)*a24*(1 - a25)*5 +
(1 - a21)*a22*(1 - a23)*a24*a25*5 +
(1 - a21)*a22*a23*(1 - a24)*(1 - a25)*3 +
(1 - a21)*a22*a23*(1 - a24)*a25*3 +
(1 - a21)*a22*a23*a24*(1 - a25)*3 +
(1 - a21)*a22*a23*a24*a25*3 +
a21*(1 - a22)*(1 - a23)*(1 - a24)*(1 - a25)*0 +
a21*(1 - a22)*(1 - a23)*(1 - a24)*a25*0 +
a21*(1 - a22)*(1 - a23)*a24*(1 - a25)*0 +
a21*(1 - a22)*(1 - a23)*a24*a25*0 +
a21*(1 - a22)*a23*(1 - a24)*(1 - a25)*0 +
a21*(1 - a22)*a23*(1 - a24)*a25*0 +
a21*(1 - a22)*a23*a24*(1 - a25)*0 +
a21*(1 - a22)*a23*a24*a25*0 +
a21*a22*(1 - a23)*(1 - a24)*(1 - a25)*0 +
a21*a22*(1 - a23)*(1 - a24)*

In [13]:
#minimize total resource consumption
costs = list()
print(rpa_df)
for app in appList:
    cost = a2n_df.loc[app]*(rpa_df.loc[:, app]).item()
    #print("cost %s" %cost)
    costs.append(sum(cost))
print(costs)
total_cost = sum(costs)
print(type(total_cost))
print(total_cost.is_real())
print(total_cost.is_int())

        APP1  APP2
memory   256   128
[1280, 0 + a21*128 + a22*128 + a23*128 + a24*128 + a25*128]
<class 'z3.ArithRef'>
True
False


# The optimizer

## Normalize

In [14]:


min_latency = 0
max_latency = max([sum(n2n_df.loc[node, :]) for node in nodeList])
scale_latency = 1/(max_latency - min_latency)
print("max latency: %s" %max_latency)
print("scale latency: %s " %scale_latency)
print(max_latency * scale_latency)

    
#min_rsrc = in nlappList
max_rsrc = max([sum(rpa_df.loc[rsrc, :]) for rsrc in resourceTypes])*len(nodeList)
print("max_rsrc: %s" %max_rsrc)
min_rsrc = rpa_df.loc['memory', 'APP2' ] + 5*rpa_df.loc['memory', 'APP1']
scale_rsrc = 1/(max_rsrc - min_rsrc)
print("min_rsrc: %s" %min_rsrc)
print("scale rsrc: %s " %scale_rsrc)
print("max_rsrc * scale = %s" %(max_rsrc * scale_rsrc))
print("type of scale_rsrc: %s" %(type(scale_rsrc)))

print("total_cost %s " %(total_cost.__rmul__(scale_rsrc)))



max latency: 30
scale latency: 0.0333333333333 
1.0
max_rsrc: 1920
min_rsrc: 1408
scale rsrc: 0.001953125 
max_rsrc * scale = 3.75
type of scale_rsrc: <class 'numpy.float64'>
total_cost 1/512*
(1280 + 0 + a21*128 + a22*128 + a23*128 + a24*128 + a25*128) 


In [15]:
def run_optimize(p):    
    opt = z3.Optimize()
    constraints = a2n_domain + rsrc_constraint + mdd
    #opt.add(constraints)
    opt.assert_exprs(constraints)
    p = p

    opt.minimize(p*latency_obj*scale_latency + (1-p)*total_cost*scale_rsrc)
    #opt.minimize(total_cost)
    #opt.maximize(m)
    opt.check()

    opt_model = opt.model()
    #print("optimization model \n %s \n" %opt_model)

    a2n_df_r = a2n_df.copy()
    for app in nlappList:
        #print('App %s' %app)
        for node in nodeList:
            #print('Node %s' %node)
            a2n_df_r.loc[app, node] = opt_model.eval(a2n_df.loc[app, node])
    #for app in appList:
    #    for node in nodeList:
    #        if a2n_df.loc[app,node] in opt_model:
    #            print (a2n_df.loc[app,node])

    #print("app placement \n %s \n" %a2n_df_r)  

    #Verify the results. 
    #print("latency cost: %s" %opt_model.eval(latency_obj*p))
    #print("resource cost: %s" %opt_model.eval(total_cost))
    return a2n_df_r
    

a2n_df_r = run_optimize(.5)
pprint.pprint(a2n_df_r)

     FSSN2 FSSN4 FSSN1 FSSN3 FSSN5
APP1     1     1     1     1     1
APP2     0     0     1     0     0


In [16]:
# Compare alternative to networkx
#https://plot.ly/python/igraph-networkx-comparison/
import networkx as nx
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from ipywidgets import interact
from ipywidgets import widgets

In [17]:
def placeStations(G):    
    for node1 in nodeList:
        #print("node1 is %s" %node1)
        G.add_node(node1, attr_dict=dict(rpn_df.loc[:, node1]))
        #print("G.nodes: %s" %G.nodes())
        id1 = data[node1]['id']
        for node2 in nodeList:
            id2 = data[node2]['id']
            if id1 != id2:            
                G.add_edge(node1, node2)  
    #circular_layout(G, dim=2, scale=1, center=None)
    pos = nx.circular_layout(G)
    #https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.classes.function.set_node_attributes.html
    nx.set_node_attributes(G, 'pos', pos)
    nx.set_node_attributes(G, 'color', 65)
    return G

In [18]:
G = nx.Graph()
G = placeStations(G)

print("\nGraph Nodes:")
pprint.pprint(G.nodes(data=True), width=90)
print("\nG.edges: ")
pprint.pprint(G.edges())


Graph Nodes:
[('FSSN2', {'color': 65, 'memory': 512, 'pos': array([ 1.,  0.])}),
 ('FSSN4', {'color': 65, 'memory': 512, 'pos': array([ 0.30901699,  0.95105652])}),
 ('FSSN1', {'color': 65, 'memory': 1024, 'pos': array([-0.80901699,  0.58778525])}),
 ('FSSN3', {'color': 65, 'memory': 512, 'pos': array([-0.80901699, -0.58778525])}),
 ('FSSN5', {'color': 65, 'memory': 512, 'pos': array([ 0.30901699, -0.95105652])})]

G.edges: 
[('FSSN2', 'FSSN3'),
 ('FSSN2', 'FSSN4'),
 ('FSSN2', 'FSSN1'),
 ('FSSN2', 'FSSN5'),
 ('FSSN4', 'FSSN3'),
 ('FSSN4', 'FSSN1'),
 ('FSSN4', 'FSSN5'),
 ('FSSN1', 'FSSN3'),
 ('FSSN1', 'FSSN5'),
 ('FSSN3', 'FSSN5')]


In [19]:
def annotateEdges(G):    
    # The normal edge annotations are on one of the edge endpoints
    # This is to place an annotation on the middle of the each edge
    edge_annotations = go.Annotations()
    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        mx, my = [(x1+x0)*.5,(y1+y0)*.5]
        edge_annotations.append(go.Annotation(
            x = mx,
            y = my, 
            xref='x',
            yref='y',
            text = n2n_df.loc[edge]))
        #print("edge: %s lat: %s" %(edge, n2n_df.loc[edge]))
        #print("mx: %s, my: %s" %(mx, my))
        #print("mx: %s, my: %s" %(mx, my))  
    return(edge_annotations)

edge_annotations = annotateEdges(G)

In [20]:
def placeApps(G, a2n_df_r):
    appColor = {}
    for node in nodeList:
        #print("\n before app Graph Edges:")
        #pprint.pprint(G.edges())
        appG = nx.Graph()
        numApps = 1 #used to increment color when there is a new app
        
        for app in appList:
            #---- Check type of a2n matrix entries. If they are a z3 type convert to an int.----
            if type(a2n_df_r.loc[app, node]) ==z3.RatNumRef:
                val = (a2n_df_r.loc[app, node]).as_fraction()
            else:
                val = a2n_df_r.loc[app, node]
            #---COLOR APPS UNIQUELY---
            if val == 1:
                if app in appColor:
                    color = appColor[app]
                else:
                    color = 65+10*numApps
                    appColor[app] = color
                #---Add appnodes their edges to the station node---
                appG.add_node(app+node, attr_dict=dict(rpa_df.loc[:, app]), color=color)
                appG.add_edge(node, app+node) #adding the edge also adds the node
            numApps += 1
        #---Place apps---            
        app_pos = nx.circular_layout(appG, scale=.05, center=G.node[node]['pos'])
        nx.set_node_attributes(appG, 'pos', app_pos)
        #---combine app graph with station graph            
        G = nx.compose(appG, G) # Attributes from H take precedent over attributes from G.             
        #print("\n node %s" %node)
        #pprint.pprint(G.nodes(data=True), width=120)
    return G

G = nx.Graph()
G = placeStations(G)
edge_annotations = annotateEdges(G)
G = placeApps(G, run_optimize(.6))
#print("\nGraph Nodes:")
#pprint.pprint(G.nodes(data=True), width=100)
#print("\n number of Graph Edges:")
#pprint.pprint(len(G.edges()))

In [21]:
def nodeTrace(G):
    node_trace = go.Scatter(
            x=[],
            y=[],
            text=[],
            mode='markers+text',
            hoverinfo='text',
            textposition='bottom',
            marker=go.Marker(
                showscale=True,
                # colorscale options
                # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
                colorscale='RdBu',
                reversescale=True,
                color=[],
                size=[],
                #colorbar=dict(
                #    thickness=15,
                #    title='Node Connections',
                #    xanchor='left',
                #    titleside='right'
                #),
                #line={'width':10}
            )
        )

    #thickness = 1
    for node in reversed(sorted(G.nodes())):
        #print(node)
        #print(G.node[node]['pos'])
        x, y = G.node[node]['pos']
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        degree = nx.degree(G, node)
        node_info = node + ": " + str(G.node[node]['memory'])
        node_trace['text'].append(node_info)
        node_trace['marker']['size'].append(15*degree)
        node_trace['marker']['color'].append(G.node[node]['color'])
        #node_trace['marker']['line'].append(thickness)
        #thickness = thickness + 1
    return node_trace
    
G = nx.Graph()
G = placeStations(G)
edge_annotations = annotateEdges(G)
G = placeApps(G, run_optimize(.5))
node_trace = nodeTrace(G)
pprint.pprint(node_trace, width=100)

{'hoverinfo': 'text',
 'marker': {'color': [65, 65, 65, 65, 65, 85, 75, 75, 75, 75, 75],
            'colorscale': 'RdBu',
            'reversescale': True,
            'showscale': True,
            'size': [75, 75, 75, 75, 90, 15, 15, 15, 15, 15, 15]},
 'mode': 'markers+text',
 'text': ['FSSN5: 512',
          'FSSN4: 512',
          'FSSN3: 512',
          'FSSN2: 512',
          'FSSN1: 1024',
          'APP2FSSN1: 128',
          'APP1FSSN5: 256',
          'APP1FSSN4: 256',
          'APP1FSSN3: 256',
          'APP1FSSN2: 256',
          'APP1FSSN1: 256'],
 'textposition': 'bottom',
 'type': 'scatter',
 'x': [0.30901699437494723,
       0.30901699437494745,
       -0.80901699437494756,
       1.0,
       -0.80901699437494734,
       -0.83401699437494736,
       0.25901699437494724,
       0.25901699437494746,
       -0.85901699437494761,
       0.94999999999999996,
       -0.7590169943749473],
 'y': [-0.95105651629515364,
       0.95105651629515353,
       -0.58778525229247303,


In [22]:
def edgeTrace(G):
    #Properties for the edges to be plotted. 
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=go.Line(width=0.5,color='#888'),
        hoverinfo='text',
        mode='lines+text',
        text=[],
        #textposition='middleright'
    )

    #---Build edge traces to put in figure.--- 
    for edge in G.edges():
        #print(edge)
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_info = edge
        edge_trace['text'].append(edge_info)
        edge_trace['x'] += [x0, x1]
        edge_trace['y'] += [y0, y1]
    return edge_trace

G = nx.Graph()
G = placeStations(G)
edge_annotations = annotateEdges(G)
G = placeApps(G, run_optimize(.9))
node_trace = nodeTrace(G)
edge_trace = edgeTrace(G)
print("\n number of edges %s\n" %len(edge_trace['x']))
pprint.pprint(edge_trace, width=150)


 number of edges 40

{'hoverinfo': 'text',
 'line': {'color': '#888', 'width': 0.5},
 'mode': 'lines+text',
 'text': [('APP1FSSN1', 'FSSN1'),
          ('FSSN1', 'FSSN2'),
          ('FSSN1', 'FSSN4'),
          ('FSSN1', 'FSSN3'),
          ('FSSN1', 'FSSN5'),
          ('FSSN1', 'APP2FSSN1'),
          ('APP1FSSN4', 'FSSN4'),
          ('APP1FSSN3', 'FSSN3'),
          ('APP1FSSN2', 'FSSN2'),
          ('FSSN5', 'FSSN2'),
          ('FSSN5', 'FSSN4'),
          ('FSSN5', 'FSSN3'),
          ('FSSN5', 'APP2FSSN5'),
          ('FSSN5', 'APP1FSSN5'),
          ('FSSN2', 'FSSN4'),
          ('FSSN2', 'APP2FSSN2'),
          ('FSSN2', 'FSSN3'),
          ('FSSN4', 'APP2FSSN4'),
          ('FSSN4', 'FSSN3'),
          ('APP2FSSN3', 'FSSN3')],
 'type': 'scatter',
 'x': [-0.7590169943749473,
       -0.80901699437494734,
       -0.80901699437494734,
       1.0,
       -0.80901699437494734,
       0.30901699437494745,
       -0.80901699437494734,
       -0.80901699437494756,
       -0.8090169

In [23]:
def plot(G, edgeAnnotations, traces):
    fig = go.Figure(data=go.Data(traces),
                 layout=go.Layout(
                     title='<br>App placement graph',
                     titlefont=dict(size=16),
                     showlegend=False,
                     hovermode='closest',
                     margin=dict(b=20,l=5,r=5,t=40),
                     #annotations=[ dict(
                     #    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                     #    showarrow=False,
                     #    xref="paper", yref="paper",
                     #    x=0.005, y=-0.002 ) ],
                     annotations=edgeAnnotations,
                     xaxis=go.XAxis(showgrid=False, zeroline=False, showticklabels=False),
                     yaxis=go.YAxis(showgrid=False, zeroline=False, showticklabels=False),
                     #sliders = sliders
                 ))

    py.iplot(fig, filename='networkx')
    


In [24]:
def test(p):
    solution = run_optimize(p)
    print("solution: %s" %solution)
    G = nx.Graph()
    G1 = placeStations(G)
    #print("G1:")
    #pprint.pprint(sorted(G1.nodes(data=True)), width=120)
    edgeAnnotations = annotateEdges(G1)
    #print(" \n edgeAnnotations")
    #pprint.pprint(edgeAnnotations, width=150)    
    G2 = placeApps(G1, solution)
    #print("G2: ")
    #pprint.pprint(sorted(G2.nodes(data=True)), width=120)
    nt = nodeTrace(G2)
    et = edgeTrace(G1)
    traces = [et, nt]
    G2.remove_edges_from(G2.edges())
    plot(G2, edgeAnnotations, traces)
    
    #print("\nG.edges: ")
    #pprint.pprint(sorted(G2.edges()))

test(.5)

solution:      FSSN2 FSSN4 FSSN1 FSSN3 FSSN5
APP1     1     1     1     1     1
APP2     0     0     1     0     0


In [25]:
p = widgets.FloatSlider(min=0, max=1, step=0.1, value=.6)

# In case we want to try different interact parameters. 
#https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html
interact(test, p=p)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.test>

In [26]:
#import networx to color graph
# 5 nodes
# primary deployed everywhere
#weight optimization criteria
#visualize
#write up

In [27]:
#minimize latency equation
# App is on FSSN2 and Dependency is deployed once
m21 = a2n_df.loc['APP1','FSSN2'] \
    * (a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1']]).item()

m22 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    * (    a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN2']]).item()
        
m23 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    * (    a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN3']]).item()

# App is on FSSN2 and Dependency is deployed twice
m21_22 = a2n_df.loc['APP1','FSSN2'] \
    *     (a2n_df.loc['APP2','FSSN1']) \
    *     (a2n_df.loc['APP2','FSSN2']) \
    * (1 - a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN2']]).item()
    
m21_23 = a2n_df.loc['APP1','FSSN2'] \
    *     (a2n_df.loc['APP2','FSSN1']) \
    * (1 - a2n_df.loc['APP2','FSSN2']) \
    *     (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()
    
m22_23 = a2n_df.loc['APP1','FSSN2'] \
    * (1 - a2n_df.loc['APP2','FSSN1']) \
    *     (a2n_df.loc['APP2','FSSN2']) \
    *     (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN2'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()

# App is on FSSN2 and Dependency is deployed three times    
m21_22_23 = a2n_df.loc['APP1','FSSN2'] \
    * (a2n_df.loc['APP2','FSSN1']) \
    * (a2n_df.loc['APP2','FSSN2']) \
    * (a2n_df.loc['APP2','FSSN3']) \
    * min([n2n_df.loc['FSSN2','FSSN1'], \
           n2n_df.loc['FSSN2','FSSN2'], \
           n2n_df.loc['FSSN2','FSSN3']]).item()

m = m21+m22+m23+m21_22+m21_23+m22_23+m21_22_23
m

#a2n_df.loc[app,node] * a2n_df.loc[dep,node]
#a2a_df.loc[app] -> [1, 0]


1*a23*(1 - a21)*(1 - a24)*3 +
1*(1 - a23)*a21*(1 - a24)*0 +
1*(1 - a23)*(1 - a21)*a24*5 +
1*a23*a21*(1 - a24)*0 +
1*a23*(1 - a21)*a24*3 +
1*(1 - a23)*a21*a24*0 +
1*a23*a21*a24*0